# Creating test cases
This notebook takes a folder of notebooks and turns them into a jsonl file in the format human_eval expects.

In [1]:
import json
import os
import warnings

In [2]:
source_notebook_directory = './human-eval-bia/' # must end with /
# Specify the filename to save the .jsonl file
target_jsonl_filename = '../data/human-eval-bia.jsonl'

In [3]:
list_of_cases = []


# List all files in the current directory
files = os.listdir(source_notebook_directory)

# Iterate through the files and print names ending with .ipynb
for file in files:
    if file.endswith('.ipynb'):
        notebook_filename = source_notebook_directory + file
        
        # Load and parse the notebook
        with open(notebook_filename, 'r') as file:
            notebook = json.load(file)
        
        task_id = notebook_filename
        prompt = None
        canonical_solution = None
        entry_point = None
        test = None
        
        # Iterate through the cells and print the source of code cells
        for cell in notebook['cells']:
            if cell['cell_type'] == 'code':
                # Joining the lines of code for better readability
                code = ''.join(cell['source'])
                # print('\n\nCODE\n\n',code)
        
                if code.startswith('check('):
                    entry_point = code.strip().replace("check(","").replace(")","").strip()
                elif '"""' in code:
                    temp = code.split('"""')
                    canonical_solution = temp[-1]
                    temp[-1] = ""
                    prompt = '"""'.join(temp)
                elif 'def check(' in code:
                    test = code 
                elif len(code.strip()) == 0:
                    pass
                else:
                    sample = code[:20]
                    warnings.warn(f"I had issues reading a cell in {task_id} starting with ")
                    
        if prompt is None:
            warnings.warn(f"Couldn't extract prompt from {task_id}.")
        elif canonical_solution is None:
            warnings.warn(f"Couldn't extract canonical_solution from {task_id}.")
        elif entry_point is None:
            warnings.warn(f"Couldn't extract entry_point from {task_id}.")
        
        test_case = {
            'task_id':task_id,
            'prompt':prompt,
            'canonical_solution':canonical_solution,
            'entry_point':entry_point,
            'test':test
        }

        print(test_case)
        list_of_cases.append(test_case)

{'task_id': './human-eval-bia/filtering_0.ipynb', 'prompt': 'def detect_edges(image):\n    """\n    Applies an edge-detection filter to an image.\n    """', 'canonical_solution': '\n    from scipy.ndimage import sobel\n    filtered_image = sobel(image)\n    return filtered_image', 'entry_point': 'detect_edges', 'test': 'def check(candidate):\n    import numpy as np\n\n    image = np.asarray([\n        [1,1,2,2,2],\n        [1,1,2,2,2],\n        [1,1,2,2,2],\n        [1,1,2,2,2],\n        [1,1,2,2,2],\n    ])\n\n    result = candidate(image)\n    left_column = result[:,0]\n    center_columns = result[:,1:4]\n    right_column = result[:,-1]\n\n    #print("left", left_column)\n    #print("center", center_columns)\n    #print("right", right_column)\n\n    assert left_column.max() == 0 and left_column.min() == 0\n    assert center_columns.max() != 0 or center_columns.min() != 0\n    assert right_column.max() == 0 and left_column.min() == 0\n'}
{'task_id': './human-eval-bia/filtering_1.ipynb

In [4]:
# Open the file in write mode and save the dictionaries
with open(target_jsonl_filename, 'w') as file:
    for dictionary in list_of_cases:
        # Convert dictionary to a JSON formatted string and write it
        json_str = json.dumps(dictionary)
        file.write(json_str + '\n')

print(f'Data successfully saved to {target_jsonl_filename}.')

Data successfully saved to ../data/human-eval-bia.jsonl.
